In [1]:
import re
import json
import numpy as np

In [64]:
month2number = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
}

reg_exprs = [
    re.compile(r'от (\d?\d) ([а-я]+) (\d\d\d\d)'),
    re.compile(r'(\d\d)\.(\d\d)\.(\d\d\d\d)'),
    re.compile(r'\n(\d?\d) ([а-я]+) (\d\d\d\d)')
]

In [65]:
def extract_date(doc, doc_type):
    matches = []
    variants = []
    doc = doc.lower()
    for expr in reg_exprs:
        matches += expr.findall(doc)
    if matches:
#         if doc_type == 'приказ' and (matches[0][1] in month2number or len(matches[0][1]) == 2):
#             match = matches[0]
#             day = match[0] if len(match[0]) == 2 else '0' + match[0]
#             month = month2number[match[1]] if match[1] in month2number else match[1]
#             year = match[2]
#             return f'{day}.{month}.{year}'
        for match in matches:
            day = match[0] if len(match[0]) == 2 else '0' + match[0]
            month = month2number[match[1]] if match[1] in month2number else match[1]
            year = match[2]
            variants.append((day, month, year))
    date = variants[0] if variants else ''
    for variant in variants:
        if f'{variant[2]}.{variant[1]}.{variant[0]}' > f'{date[2]}.{date[1]}.{date[0]}':
            date = variant
    return f'{date[0]}.{date[1]}.{date[2]}' if date else date

def predict(test, types):
    results = []
    for doc, doc_type in zip(test, types):
        date = extract_date(doc, doc_type)
        prediction = {"type": "",
                      "date": date,
                      "number": "",
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [66]:
from eval_module import quality

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])
predicted = predict(train, true_types)
quality(predicted, labeled)

{'date_accuracy': 0.9533717724,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [67]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted_labels[ids], test_labels[ids])['date_accuracy']}")

федеральный закон: 0.9947916667
постановление: 0.9365595771
приказ: 0.8961038961
распоряжение: 0.9953488372
закон: 0.9698492462
указ: 0.9794303797


In [68]:
doc_type = 'приказ'
ids = np.where(true_types == doc_type)

y_test = [(x['date'], x['type']) for x in test_labels[ids]]
y_pred = [x['date'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0] != pred:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f.read())
        print("=====")

right value = ('26.10.2015', 'приказ')
wrong value = 27.10.2015
3‚: :‘ЭЁ*}!"
МИНИСТЕРСТВО ЮСТВННИ РОССИНСКОЙ ФЕДЕРАЦИИ

|
‚ЗАРЕГИСТРИРОВАНО Ё
’_РИСТ Рьциш"ныи Л &__@__* ;

 

МИНИСТЕРСТВО ЮСТИЦИИ РОССИЙСКОЙ ФЕДЕРАЦИИ
° - (МИНЮСТ РОССИИ)

ПРИКАЗ

Москва
26 октября 2015 г. № 248

О внесении изменения
в приказ Минюста России от 14 января 2015 г. № 5
"О распространении на работников, замещающих отдельные должности
на основании трудового договора в организациях, созданных для выполнения
задач, поставленных перед Министерством юстиции Российской
Федерации, ограничений, запретов и обязанностей,
установленных для федеральных государственных
гражданских служащих"

В соответствии со статьей 12.2 Федерального закона от 25 декабря 2008 г.
№ 273-ФЗ3 "О противодействии - коррупции" (Собрание - законодательства
Российской Федерации, 2008, № 52 (ч. 1), ст. 6228; 2011, № 29, ст. 4291, № 48%,
ст. 6730; 2012, № 50 (ч. 4), ст. 6954, № 53 (ч. 1), ст. 7605; 2013, № 19, ст. 2329, № 40
(ч. 3), ст. 5031, № 52 

# Для каждого типа документа свой извлекатор:

* указ - дата в конце 12 июня 2012 года

* постановление - дата в начале (от) 17.12.2015 г., 10 марта 2015 года 

* распоряжение - дата в начале от 27 ноября 2014 г.
